Import

In [1]:
import os
from contextlib import redirect_stdout

import sys
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

Init and read

In [2]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

df = add_water_year_column(df)

df

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


A                          CALSIM                                     \
B          BANKSEC_MAX14DAY_s0001 COREQSACDV_s0001 CO_EC_MONTH_s0001   
C                 SALINITY-APPROX             FLOW          SALINITY   
D                            1MON             1MON              1MON   
E                          L2020A           L2020A            L2020A   
F                        PER-AVER         PER-AVER          PER-AVER   
Units                    UMHOS/CM              CFS          UMHOS/CM   
1921-10-31              284.73438           0.0000         2173.8489   
1921-11-30              281.05875           0.0000         2621.5984   
1921-12-31              376.29092        4017.6848         5590.7207   
1922-01-31              623.95900           0.0000        10744.2570   
1922-02-28              416.49713           0.0000         3148.9802   
..                            ...              ...               ...   
2021-05-31              393.38092           0.0000         4102.8306   
2021-06-30              383.89062           0.0000         6367.1660   
2021-07-31              446.86966           0.0000         8060.9453   
2021-08-31              464.96625           0.0000         8370.9795   
2021-09-30              438.54697           0.0000         8845.4890   

A                                                                      \
B          C_AMR004_s0001       C_AMR004_ADD_s0001 C_AMR004_MIF_s0001   
C                 CHANNEL FLOW-ADDITIONAL-INSTREAM  FLOW-MIN-INSTREAM   
D                    1MON                     1MON               1MON   
E                  L2020A                   L2020A             L2020A   
F                PER-AVER                 PER-AVER           PER-AVER   
Units                 CFS                      CFS                CFS   
1921-10-31     2164.72950              1664.729500          500.00000   
1921-11-30     2206.79600              1706.795900          500.00000   
1921-12-31     2024.89980              1524.899800          500.00000   
1922-01-31     1704.54830              1454.548300          250.00000   
1922-02-28     5005.48340              4755.483400          250.00000   
..                    ...                      ...                ...   
2021-05-31      645.44150               380.476620          227.46918   
2021-06-30     1839.48110              1574.516100          227.46918   
2021-07-31     1755.98750              1491.022700          227.46918   
2021-08-31     1914.82500              1649.860000          227.46918   
2021-09-30      402.62833                44.663414          320.46918   

A                                                                \
B          C_CAA003_s0001 C_CAA003_CVP_s0001 C_CAA003_SWP_s0001   
C                 CHANNEL      FLOW-DELIVERY      FLOW-DELIVERY   
D                    1MON               1MON               1MON   
E                  L2020A             L2020A             L2020A   
F                PER-AVER           PER-AVER           PER-AVER   
Units                 CFS                CFS                CFS   
1921-10-31     2128.54900             0.0000         1960.62740   
1921-11-30     5519.24200          2043.4584         3475.78370   
1921-12-31     7353.68950             0.0000         7353.68950   
1922-01-31     2900.16060             0.0000         2900.16060   
1922-02-28     4465.35600             0.0000         4465.35600   
..                    ...                ...                ...   
2021-05-31      543.74133             0.0000          543.74133   
2021-06-30      300.00000             0.0000          300.00000   
2021-07-31      787.90320             0.0000          300.00000   
2021-08-31      462.63440             0.0000          300.00000   
2021-09-30      603.00000             0.0000          300.00000   

A                             ...       CALCULATED                   \
B          C_CAA003_TD_s0001  ... DEL_SOD_AG_s0021 DEL_SOD_MI_s0021   
C              FLOW-DELIVERY  ...    DE

Define vars

In [3]:
in_delta_vars = ["EM_EC_MONTH", "JP_EC_MONTH"]
export_vars = ["TRACYEC_MAX14DAY", "BANKSEC_MAX14DAY"]

Subset in-delta df and export df

In [4]:
in_delta_df = create_subset_list(df, in_delta_vars)
in_delta_df

A                     CALSIM                                      \
B          EM_EC_MONTH_s0001 JP_EC_MONTH_s0001 EM_EC_MONTH_s0002   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1921-10-31         420.35240         308.33957         420.35240   
1921-11-30         561.92170         298.96088         560.35170   
1921-12-31        1132.24480        1211.59310        1200.42440   
1922-01-31         713.35815        1374.92550         724.70953   
1922-02-28         359.12146         654.43500         345.48660   
..                       ...               ...               ...   
2021-05-31         849.52420         517.29944         838.77576   
2021-06-30        1616.85250         842.68225        1669.95700   
2021-07-31        2524.26300        1285.16800        2613.50560   
2021-08-31        2431.64900        1367.02320        2499.81080   
2021-09-30        2797.95000        1376.98610        2753.90580   

A                                                                 \
B          JP_EC_MONTH_s0002 EM_EC_MONTH_s0003 JP_EC_MONTH_s0003   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1921-10-31         308.33957         378.06560         269.90536   
1921-11-30         299.63480         478.07477         248.80586   
1921-12-31        1283.97570        1018.15460         836.14660   
1922-01-31        1500.18680         379.06232         840.27026   
1922-02-28         656.80850         214.80246         287.58038   
..                       ...               ...               ...   
2021-05-31         506.96332         875.91693         493.00034   
2021-06-30         872.56220        2112.61400         846.22380   
2021-07-31        1316.08480        2823.19630        1496.07540   
2021-08-31        1346.60570        2629.35770        1401.00840   
2021-09-30        1318.96280        2826.85130        1325.63900   

A                                                                 \
B          EM_EC_MONTH_s0004 JP_EC_MONTH_s0004 EM_EC_MONTH_s0005   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1921-10-31         403.16763         253.26294         403.16763   
1921-11-30         436.82724         245.62279         433.17044   
1921-12-31         998.01874         863.27500        1009.44750   
1922-01-31         338.66785         742.52374         380.75513   
1922-02-28         243.60349         300.92972         268.10992   
..                       ...               ...               ...   
2021-05-31        1032.20580         498.76810        1138.69620   
2021-06-30        2514.34940         973.80164        2580.97020   
2021-07-31        2750.93070        1495.42990        2718.73780   
2021-08-31        2567.57520        1231.75630        2655.68460   
2021-09-30        2621.02930        1204.72670        2633.80660   

A                             ...                                      \
B          JP_EC_MONTH_s0005  ... EM_EC_MONTH_s0016 JP_EC_MONTH_s0016   
C                   SALINITY  ...          SALINITY          SALINITY   
D                       1MON  ...              1MON              1MON   
E                     L2020A  ...

In [5]:
df_1924 = in_delta_df[in_delta_df.index.year == 1924]
df_1924

A                     CALSIM                                      \
B          EM_EC_MONTH_s0001 JP_EC_MONTH_s0001 EM_EC_MONTH_s0002   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1924-01-31        1724.01040        1787.44570        1883.39340   
1924-02-29         523.19600         876.46290         945.63495   
1924-03-31         262.47256         365.68190         470.85196   
1924-04-30         326.46110         294.10632         346.06230   
1924-05-31         675.33600         362.03330         613.37427   
1924-06-30        1524.80130         741.40210        1462.73100   
1924-07-31        2372.11430        1079.53750        2309.63920   
1924-08-31        2331.62650        1200.67900        2346.46240   
1924-09-30        2740.66530        1239.10200        2738.09030   
1924-10-31        3346.86200        1589.81920        3341.89600   
1924-11-30        4113.50340        2175.24370        4067.50120   
1924-12-31        4144.32100        2197.72140        4141.19780   

A                                                                 \
B          JP_EC_MONTH_s0002 EM_EC_MONTH_s0003 JP_EC_MONTH_s0003   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1924-01-31         1557.8167        1969.09970        1586.24850   
1924-02-29         1303.8164        1241.51800        1525.67150   
1924-03-31          828.2897         542.55250         917.93945   
1924-04-30          352.0795         360.71120         359.91025   
1924-05-31          363.2632         627.29755         336.12152   
1924-06-30          704.8734        1531.33760         587.20026   
1924-07-31         1024.8622        2473.93600        1132.47630   
1924-08-31         1171.5968        2459.83860        1249.51730   
1924-09-30         1241.0810        2862.74170        1238.39150   
1924-10-31         1601.4166        3586.87100        1606.96980   
1924-11-30         2175.6838        4121.15800        2239.40940   
1924-12-31         2196.7607        3603.13010        1881.57500   

A                                                                 \
B          EM_EC_MONTH_s0004 JP_EC_MONTH_s0004 EM_EC_MONTH_s0005   
C                   SALINITY          SALINITY          SALINITY   
D                       1MON              1MON              1MON   
E                     L2020A            L2020A            L2020A   
F                   PER-AVER          PER-AVER          PER-AVER   
Units               UMHOS/CM          UMHOS/CM          UMHOS/CM   
1924-01-31        2316.57250        1726.89600        2191.95140   
1924-02-29        1589.44170        1548.33970        1468.54220   
1924-03-31         738.52150         985.93860         672.97920   
1924-04-30         468.18817         353.08652         533.68430   
1924-05-31         717.44226         334.88153         784.19025   
1924-06-30        1795.48580         719.42110        2079.34400   
1924-07-31        2618.19120        1089.80150        2644.59800   
1924-08-31        2433.39180        1022.01330        2244.87940   
1924-09-30        2615.52510        1001.35710        2649.94730   
1924-10-31        3149.29270        1338.05940        3353.14770   
1924-11-30        4023.08600        2170.71170        4172.02000   
1924-12-31        3271.76440        1825.03540        3213.86990   

A                             ...                                      \
B          JP_EC_MONTH_s0005  ... EM_EC_MONTH_s0

In [6]:
export_df = create_subset_list(df, export_vars)
export_df

A                          CALSIM                         \
B          BANKSEC_MAX14DAY_s0001 TRACYEC_MAX14DAY_s0001   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              284.73438              357.93190   
1921-11-30              281.05875              354.92264   
1921-12-31              376.29092              432.88925   
1922-01-31              623.95900              635.65515   
1922-02-28              416.49713              465.80610   
..                            ...                    ...   
2021-05-31              393.38092              446.88086   
2021-06-30              383.89062              439.11115   
2021-07-31              446.86966              490.67210   
2021-08-31              464.96625              505.48776   
2021-09-30              438.54697              483.85828   

A                                                         \
B          BANKSEC_MAX14DAY_s0002 TRACYEC_MAX14DAY_s0002   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              284.73438              357.93190   
1921-11-30              280.69608              354.62573   
1921-12-31              400.25623              452.50967   
1922-01-31              686.99084              687.25934   
1922-02-28              419.23160              468.04480   
..                            ...                    ...   
2021-05-31              395.85843              448.90918   
2021-06-30              383.63202              438.89940   
2021-07-31              459.99188              501.41525   
2021-08-31              473.36250              512.36176   
2021-09-30              424.56873              472.41430   

A                                                         \
B          BANKSEC_MAX14DAY_s0003 TRACYEC_MAX14DAY_s0003   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              329.17500              394.31543   
1921-11-30              296.69278              367.72226   
1921-12-31              411.21050              461.47794   
1922-01-31              565.80280              588.04266   
1922-02-28              298.34177              369.07227   
..                            ...                    ...   
2021-05-31              366.52634              424.89500   
2021-06-30              373.53330              430.63160   
2021-07-31              482.44656              519.79890   
2021-08-31              493.06670              528.49360   
2021-09-30              457.05203              499.00840   

A                                                         \
B          BANKSEC_MAX14DAY_s0004 TRACYEC_MAX14DAY_s0004   
C                 SALINITY-APPROX        SALINITY-APPROX   
D                            1MON                   1MON   
E                          L2020A                 L2020A   
F                        PER-AVER               PER-AVER   
Units                    UMHOS/CM               UMHOS/CM   
1921-10-31              360.45148              419.92150   
1921-11-30              358.37662              418.22280   
1921-12-31              345.13815              407.38450   
1922-01-31              447.98657              491.58650   
1922-02-28              297.62354              368.48425   
..                            ...    

### In-Delta tier assignment:

In [7]:
def generate_salinity_tier_assignment_matrix(
    df,
    station_list=["EM", "JP"],
    thresholds={"Top": 2500, "Mid": 1600, "Low": 900},
    start_date="1921-10-01"
):
    def extract_scenario_id(colname):
        name = "_".join(colname) if isinstance(colname, tuple) else str(colname)
        match = re.search(r's\d{4}', name)
        return match.group(0) if match else None

    def extract_station_name(colname):
        name = "_".join(colname) if isinstance(colname, tuple) else str(colname)
        for st in station_list:
            if name.startswith(st + "_") or f"_{st}_" in name:
                return st
        return None

    def assign_tiers_by_scenario(df, date_series):
        tier_rows = []
        scenario_map = {}

        for col in df.columns:
            sid = extract_scenario_id(col)
            station = extract_station_name(col)
            if sid and station:
                scenario_map.setdefault(sid, {})[station] = col

        print(f"Found {len(scenario_map)} scenarios: {list(scenario_map.keys())}")

        for sid, col_dict in scenario_map.items():
            if not all(st in col_dict for st in station_list):
                print(f" Skipping {sid}: missing one or more station columns")
                continue

            df_scenario = pd.DataFrame(
                {st: df[col_dict[st]] for st in station_list},
                index=date_series
            )
            df_scenario["Year"] = df_scenario.index.year

            valid_rows = df_scenario.dropna(subset=station_list)
            if valid_rows.empty:
                print(f" Skipping {sid}: all data is NaN")
                continue

            yearly = valid_rows.groupby("Year")
            valid_years = list(yearly.groups.keys())
            total_years = len(valid_years)

            tier4_flag = False
            tier3_flag = False
            tier3_years_with_1month_over_mid = 0
            tier2_valid_years = 0
            tier1_valid_years = 0
            any_year_exceeds_mid = False

            for year, group in yearly:
                readings = {st: group[st] for st in station_list}

                if any((r > thresholds["Top"]).sum() >= 2 for r in readings.values()):
                    tier4_flag = True
                    break

                if any((r > thresholds["Mid"]).sum() >= 2 for r in readings.values()):
                    tier3_flag = True

                if any((r > thresholds["Mid"]).any() for r in readings.values()):
                    tier3_years_with_1month_over_mid += 1


                if any((r > thresholds["Mid"]).any() for r in readings.values()):
                    any_year_exceeds_mid = True
                else:
                    in_range_counts = [((r >= thresholds["Low"]) & (r <= thresholds["Mid"])).sum() for r in readings.values()]
                    if all(count >= 10 for count in in_range_counts):
                        tier2_valid_years += 1


                if all(((r < thresholds["Low"]).sum() == 12) for r in readings.values()):
                    tier1_valid_years += 1

            if total_years == 0:
                print(f" Scenario {sid}: No valid years with complete data.")
                continue

            if tier4_flag:
                tier = 4
            elif tier3_flag or (tier3_years_with_1month_over_mid / total_years > 0.05):
                tier = 3
            elif not any_year_exceeds_mid and (tier2_valid_years / total_years >= 0.95):
                tier = 2
            elif tier1_valid_years / total_years >= 0.95:
                tier = 1
            else:
                tier = None
                print(f" Scenario {sid} did not match any tier.")
                print(f"   Summary: tier3_flag={tier3_flag}, tier3_pct={tier3_years_with_1month_over_mid / total_years:.2f}, "
                      f"tier2_pct={tier2_valid_years / total_years:.2f}, tier1_pct={tier1_valid_years / total_years:.2f}, "
                      f"any_year_exceeds_mid={any_year_exceeds_mid}")
                continue

            print(f"→ Scenario {sid} assigned Tier {tier}")
            tier_rows.append({
                "Scenario": sid,
                "Tier": tier
            })

        return pd.DataFrame(tier_rows, columns=["Scenario", "Tier"])

    df = df.copy()
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df.index = pd.date_range(start=start_date, periods=len(df), freq="MS")

    date_series = df.index
    tier_df = assign_tiers_by_scenario(df, date_series)

    if tier_df.empty:
        print(" No valid scenario-station pairs were found.")
        return pd.DataFrame(columns=["Tier"])

    return tier_df.set_index("Scenario")


In [8]:
tier_df = generate_salinity_tier_assignment_matrix(
    df=in_delta_df,
    station_list=["EM", "JP"],
    thresholds={"Top": 2500, "Mid": 1600, "Low": 900},
    start_date="1921-10-01"
)

Found 20 scenarios: ['s0001', 's0002', 's0003', 's0004', 's0005', 's0006', 's0007', 's0008', 's0009', 's0010', 's0011', 's0012', 's0013', 's0014', 's0015', 's0016', 's0018', 's0019', 's0020', 's0021']
→ Scenario s0001 assigned Tier 4
→ Scenario s0002 assigned Tier 4
→ Scenario s0003 assigned Tier 4
→ Scenario s0004 assigned Tier 4
→ Scenario s0005 assigned Tier 4
→ Scenario s0006 assigned Tier 4
→ Scenario s0007 assigned Tier 4
→ Scenario s0008 assigned Tier 4
→ Scenario s0009 assigned Tier 4
→ Scenario s0010 assigned Tier 4
→ Scenario s0011 assigned Tier 4
→ Scenario s0012 assigned Tier 4
→ Scenario s0013 assigned Tier 4
→ Scenario s0014 assigned Tier 4
→ Scenario s0015 assigned Tier 4
→ Scenario s0016 assigned Tier 4
→ Scenario s0018 assigned Tier 4
→ Scenario s0019 assigned Tier 4
→ Scenario s0020 assigned Tier 4
→ Scenario s0021 assigned Tier 4


In [9]:
tier_df

,Tier
Scenario,
s0001,4
s0002,4
s0003,4
s0004,4
s0005,4
s0006,4
s0007,4
s0008,4
s0009,4


In [10]:
tier_df.to_csv("in_delta_salinity_tiers.csv")


In [11]:
tier_export_df = generate_salinity_tier_assignment_matrix(
    df=export_df,
    station_list=["BANKSEC", "TRACYEC"],
    thresholds={"Top": 2500, "Mid": 1600, "Low": 900},
    start_date="1921-10-01"
)

Found 20 scenarios: ['s0001', 's0002', 's0003', 's0004', 's0005', 's0006', 's0007', 's0008', 's0009', 's0010', 's0011', 's0012', 's0013', 's0014', 's0015', 's0016', 's0018', 's0019', 's0020', 's0021']
→ Scenario s0001 assigned Tier 1
→ Scenario s0002 assigned Tier 1
→ Scenario s0003 assigned Tier 1
→ Scenario s0004 assigned Tier 1
→ Scenario s0005 assigned Tier 1
→ Scenario s0006 assigned Tier 1
→ Scenario s0007 assigned Tier 1
→ Scenario s0008 assigned Tier 1
→ Scenario s0009 assigned Tier 1
→ Scenario s0010 assigned Tier 1
→ Scenario s0011 assigned Tier 1
→ Scenario s0012 assigned Tier 1
→ Scenario s0013 assigned Tier 1
→ Scenario s0014 assigned Tier 1
→ Scenario s0015 assigned Tier 1
→ Scenario s0016 assigned Tier 1
→ Scenario s0018 assigned Tier 1
→ Scenario s0019 assigned Tier 1
→ Scenario s0020 assigned Tier 1
→ Scenario s0021 assigned Tier 1


In [12]:
tier_export_df.to_csv("delta_export_salinity_tiers.csv")

### Save Files

In [13]:
def find_calsim_base_path(start_path, target_folder="CalSim3_Model_Runs"):
    # Go up twice to reach DSP, then look for sibling folder
    current_path = os.path.abspath(start_path)
    dsp_root = os.path.dirname(os.path.dirname(current_path))  # notebooks → coeqwal → DSP
    candidate = os.path.join(dsp_root, target_folder)
    if os.path.isdir(candidate):
        return candidate
    raise FileNotFoundError(f"{target_folder} not found alongside {dsp_root}")


base_dir = os.path.abspath(".")
calsim_base_path = find_calsim_base_path(base_dir)

salinity_output_dir = os.path.join(
    calsim_base_path,
    "Scenarios",
    "Performance_Metrics",
    "Tiered_Outcome_Measures",
    "Salinity"
)
os.makedirs(salinity_output_dir, exist_ok=True)

salinity_output_path = os.path.join(salinity_output_dir, "InDeltaTierAssignment.csv")
tier_df.to_csv(salinity_output_path, index=True)

salinity_output_path = os.path.join(salinity_output_dir, "ExportTierAssignment.csv")
tier_export_df.to_csv(salinity_output_path, index=True)


In [14]:
print(salinity_output_dir)

C:\Dino\COEQWAL\COEQWAL_GIT\CalSim3_Model_Runs\Scenarios\Performance_Metrics\Tiered_Outcome_Measures\Salinity


In [15]:
print("Done!")

Done!
